In [ ]:

!pip install lemminflect
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import string
from  nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

     |████████████████████████████████| 778kB 4.2MB/s 


In [ ]:
dataset_columns = ["target", "id", "date", "flag", "user","text"]
dataset_encoding = "ISO-8859-1"
data = pd.read_csv('twitter.csv', encoding= dataset_encoding, names = dataset_columns, engine  = "python", error_bad_lines= False)
data.head()
X = data.iloc[:,[5]]
Y = data.iloc[:,0]
Y[Y == 4]=1


Skipping line 456650: unexpected end of data
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
num_missing_desc = data.isnull().sum()[2]
print('numbr of missing values', str(num_missing_desc))
data = data.dropna()
tag_cleaning_re= "@\S+"
#tagremoval
X['text'] = X['text'].map(lambda x : re.sub(tag_cleaning_re,'',str(x)))
X['text'] = X['text'].map(lambda x : x.lower()) #lowercase conversion
X['text'] = X['text'].map(lambda x : re.sub(r'\d+','',x)) # number removaal

text_cleaning_link = "https?:\S+|http?:\S|[^A-Za-z0-9]+"
X['text'] = X['text'].map(lambda x : re.sub('text_cleaning_link', "" , str(x)))
X['text'] = X['text'].map(lambda x : x.translate(x.maketrans('','', string.punctuation)))  #remove punctuation
X['text'] = X['text'].map(lambda x : x.strip())  # remove whitespace
X['text'] = X['text'].map(lambda x : word_tokenize(x))
X['text'] = X['text'].map(lambda x : [word for word in x if word.isalpha()])  #to remove alphabets
stop_word = set(stopwords.words('english'))
X['text'] = X['text'].map(lambda x : [w for w in x if not w in stop_word]) # stop word filterization 

lem = WordNetLemmatizer
X['text'] = X['text'].map(lambda x: [lem.lemmatize(word,'v') for word in x])

# Turn lists back to string
X['text'] = X['text'].map(lambda x: ' '.join(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


NameError: ignored

In [ ]:
X.head(10)

,text
0,v v v v v v v v v v
1,v v v v v v v v v v v v
2,v v v v v v v v v
3,v v v v v v
4,v v v v v
5,v v
6,v v
7,v v v v v v v v v v v v v
8,v v
9,v v


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_test,y_tarin = train_test_split(X,Y,test_size = 0.2,random_state =42)
print("Train size", len(X_train))
print("test size", len(X_test))

Train size 365319
test size 91330


In [ ]:
import gensim
from gensim.models import word2vec
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10
document = [_text.split() for _text in str(X_test.text)]
w2v_model = word2vec.Word2Vec(size = W2V_SIZE, window = W2V_WINDOW, min_count = W2V_MIN_COUNT, workers = 8)
w2v_model.build_vocab(document)



In [ ]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("vocab size", vocab_size)

vocab size 1


In [ ]:
w2v_model.train(document, total_examples = len(document),epochs= W2V_EPOCH)

(80, 5568)

In [ ]:
w2v_model.most_similar("hate")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [ ]:
#tokenizing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequence
from keras.model import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.utils.np_utils import to_categorial

max_sequence_length = 300 #length of tweet
embedding_dim = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_test.text)
word_index = tokenizer.word_index
vocab_size = len(word_index)
print('found %s unique tokens', len(word_index))

#convert the data to sequence 
X_train_padded = tokenizer_texts_sequence(X_train.text)
X_train_padded = pad_sequence(X_train_padded, maxlen=max_sequence_length)
print('Shape of data tensor:', X_train_padded.shape)

ImportError: ignored

In [ ]:
import pickle
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
#matrix
embedding_matrix= np.zeros((vocab_size+1,W2V_SIZE))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.mv:
    embedding_matterix[i] = w2v_model.mv[word]
print(embedding_matrix.shape)


In [ ]:
from keras 
model = Sequential()
model.add(Embedding(vocab_size+1, W2V_SiZE,weights = [embedding_matrix], input_length = max_sequence_length, trainable = False))
model.add(Dropout(.5))
model.add(LSTM(100, dropout(02), recurrent_dropout = .2)
model.add(Dense(1,activation = 'Sigmoid'))

model.summary()

model.compile(loss = 'binary_crossentropy', optimizer = "adam", matrics = ['accuracy'])


 

In [ ]:
'''
# Training 
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

BATCH_SIZE = 1024
history = model.fit(X_train_padded, y_train,
                    batch_size=512,
                    epochs=5,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)
'''

In [ ]:
from keras.models import load_model #loadmodel
model = load_model('Sentiment_LSTM_model.h5')

In [ ]:
import matplotlib.pyplat as plt
X_test_padded = tokenizer.texts_to_sequences(X_test.text)
X_test_padded = pad_sequences(X_test_padded,maxlen =max_sequence_length)
score = model.evaluate(X_test_padded,y_test,batch_size=512)
print("Accuracy:", score[1])
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,acc,'b', label = 'Training acc')
plt.plot(epochs,val_acc,'r',label = 'validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss,'b', label= 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'validation loss')
plt.title('training and validation loss')
plt.legend()

plt.show()

In [ ]:
def predict(text,include_neutral = True):
  x_test = pad_sequence(tokenizer.texts_to_sequences([text]),maxlen = max_sequence_length)
  score = model.predict([x_text])[0]
  if(score >=0.4 and score<= 0.6):
    label = "neutral"
  if(score<=0.4):
    label ="Negative"
  if(score>= 0.6):
    label = "Positive"
  return { "label" : label ,"score" : float(score)}

In [ ]:
predict("i hate laptop")